lighteternal/fact-or-opinion-xlmr-el

In [3]:
import torch
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def classify_sentence(model_name, sentence):
    """
    Classify a sentence as FACT or OPINION using a pre-trained model.
    
    Args:
        model_name (str): Name of the pre-trained model to use
        sentence (str): Sentence to classify
        
    Returns:
        dict: Classification results including scores, classification, and inference time
    """
    # Set device (GPU if available, otherwise CPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Load tokenizer and model
    print(f"Loading model: {model_name} on {device}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.to(device)
    model.eval()
    
    # Measure inference time
    start_time = time.time()
    
    # Tokenize input and move to device
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get probabilities
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Assuming: 0 = objective/fact, 1 = subjective/opinion
    obj_score = probs[0][1].item()
    subj_score = probs[0][0].item()
    classification = "FACT" if obj_score > subj_score else "OPINION"
    
    end_time = time.time()
    inference_time = end_time - start_time
    
    # Prepare results
    result = {
        "sentence": sentence,
        "objective_score": obj_score,
        "subjective_score": subj_score,
        "classification": classification,
        "inference_time_seconds": inference_time
    }
    
    return result

# Main execution
if __name__ == "__main__":
    # Pre-trained model to use
    model_name = "lighteternal/fact-or-opinion-xlmr-el"
    
    # Sample sentence to classify
    sentence = "In my opinion, this is the best movie ever made."
    sentence = "I'm so happy about this new opportunity!"
    sentence = "Paris is the capital of France"
    
    # Classify the sentence
    result = classify_sentence(model_name, sentence)
    
    # Print results
    print("\nClassification Results:")
    print(f"Sentence: {result['sentence']}")
    print(f"Objective Score: {result['objective_score']:.4f}")
    print(f"Subjective Score: {result['subjective_score']:.4f}")
    print(f"Classification: {result['classification']}")
    print(f"Inference Time: {result['inference_time_seconds']:.4f} seconds")

Loading model: lighteternal/fact-or-opinion-xlmr-el on cpu

Classification Results:
Sentence: Paris is the capital of France
Objective Score: 0.9981
Subjective Score: 0.0019
Classification: FACT
Inference Time: 0.1420 seconds
